In [2]:
import json
import numpy as np
import math
from tabulate import tabulate
from compute_tools import *
from view_tools import *

## Get Predictions

In [5]:
# gt_file='/home/ec2-user/segmenter/ETTDATA/Test/downsized/MIMIC/annotations/test_annotations_enl5.json'
# with open(gt_file) as f:
#     gt_data = json.load(f)

# gt_labels = []
# for anno in gt_data['annotations']:
#     bbox = anno['bbox']
#     x = bbox[0] + bbox[2] / 2
#     y = bbox[1] + bbox[3] / 2
#     gt_labels.append([anno['image_id'], anno['category_id'], x, y])
# gt_df = pd.DataFrame(gt_labels, columns=['image_id', 'category_id', 'x', 'y'])
# gt_df.to_csv("labels/gt_labels_mimic.csv", index=False)    

gt_file='/home/ec2-user/segmenter/ETTDATA/train_mimic_only/annotations/val_annotations_enl5.json'
with open(gt_file) as f:
    gt_data = json.load(f)

gt_labels = []
for anno in gt_data['annotations']:
    bbox = anno['bbox']
    x = bbox[0] + bbox[2] / 2
    y = bbox[1] + bbox[3] / 2
    gt_labels.append([anno['image_id'], anno['category_id'], x, y])
gt_df = pd.DataFrame(gt_labels, columns=['image_id', 'category_id', 'x', 'y'])
gt_df.to_csv("labels/gt_labels_mimic_only_val.csv", index=False)   

In [8]:
def get_max_pred_bbox(pred_file='mask_rcnn_test-dev_results.bbox.json'):
    with open(pred_file) as f:
        data = json.load(f)
        max_score = {}
        for pred in data:
            key = (pred['image_id'], pred['category_id'])
            if key not in max_score.keys():
                max_score[key] = [pred]
            elif pred['category_id'] == 3046 or pred['category_id'] == 3047:
                if pred['score'] > max_score[key][0]['score']:
                    max_score[key] = [pred]
            # elif pred['category_id'] == 3048:
            #     curr = max_score[key]
            #     if len(curr) < 2:
            #         curr.append(pred)
            #     elif pred['score'] > min(curr[0]['score'], curr[1]['score']):
            #             if curr[0]['score'] < curr[1]['score']:
            #                 curr[0] = pred
            #             else:
            #                 curr[1] = pred
    return max_score

In [32]:
def get_labels(max_score, thres=0.3):
    pred_labels = []
    for item in max_score:
        item=item[0]
        if item['score'] > thres:
            x = item['bbox'][0] + item['bbox'][2]/2
            y = item['bbox'][1] + item['bbox'][3]/2
            pred_labels.append([item['image_id'], item['category_id'], x, y])
    pred_labels = pd.DataFrame(data=pred_labels, columns=["image_id", "category_id", "x", "y"])
    return pred_labels

In [36]:
pred_file='/home/ec2-user/segmenter/InternImage/detection/test_results/mimic-only-mimic.bbox.json'
max_score = get_max_pred_bbox_3cls(pred_file)
max_score = list(max_score.values())
pred_mimic = get_labels(max_score, thres=0.3)
pred_mimic.to_csv("labels/train_mimic_only/pred_labels_mimic.csv", index=False)

pred_file='/home/ec2-user/segmenter/InternImage/detection/test_results/sampled-data-mimic.bbox.json'
max_score = get_max_pred_bbox_3cls(pred_file)
max_score = list(max_score.values())
pred_mimic = get_labels(max_score, thres=0.3)
pred_mimic.to_csv("labels/sampled_data_split/pred_labels_mimic.csv", index=False)

pred_file='/home/ec2-user/segmenter/InternImage/detection/test_results/all-data-mimic.bbox.json'
max_score = get_max_pred_bbox_3cls(pred_file)
max_score = list(max_score.values())
pred_mimic = get_labels(max_score, thres=0.3)
pred_mimic.to_csv("labels/all_data_split/pred_labels_mimic.csv", index=False)

In [35]:
pred_file='/home/ec2-user/segmenter/InternImage/detection/test_results/mimic-only-ranzcr.bbox.json'
max_score = get_max_pred_bbox_3cls(pred_file)
max_score = list(max_score.values())
pred_ranzcr = get_labels(max_score, thres=0.3)
pred_ranzcr.to_csv("labels/train_mimic_only/pred_labels_ranzcr.csv", index=False)

pred_file='/home/ec2-user/segmenter/InternImage/detection/test_results/sampled-data-ranzcr.bbox.json'
max_score = get_max_pred_bbox_3cls(pred_file)
max_score = list(max_score.values())
pred_ranzcr = get_labels(max_score, thres=0.3)
pred_ranzcr.to_csv("labels/sampled_data_split/pred_labels_ranzcr.csv", index=False)

pred_file='/home/ec2-user/segmenter/InternImage/detection/test_results/all-data-ranzcr.bbox.json'
max_score = get_max_pred_bbox_3cls(pred_file)
max_score = list(max_score.values())
pred_ranzcr = get_labels(max_score, thres=0.3)
pred_ranzcr.to_csv("labels/all_data_split/pred_labels_ranzcr.csv", index=False)

## Evaluate

In [30]:
import sys
sys.path.append('../../ETT_Evaluation')
from eval.eval import ETTEvaler

In [58]:
# evaler = ETTEvaler("labels/gt_labels_mimic.csv", "labels/all_data_split/pred_labels_mimic.csv", 1280)
evaler = ETTEvaler("labels/gt_labels_ranzcr.csv", "labels/all_data_split/pred_labels_ranzcr.csv", 1280)
tp, fp, tn, fn, distances = evaler.gt_pred_distance(target='tip')
evaler.get_metrics(tp, fp, tn, fn, distances, cat='tip')

**************************************
Max tip distance:  6.1127393796459115
Min tip distance:  0.012614575429264537
Mean tip distance:  0.40943207517617664
Std tip distance:  0.8553277391190128
Median tip distance:  0.20464243434429644
**************************************
Precision:  1.0
Recall:  0.6608695652173913
Sensitivity:  0.6608695652173913
Specificity:  1.0
F1 score:  0.7958115183246073
+--------+-------+----------+
|        |   tip |   No tip |
+========+=======+==========+
| tip    |    76 |        0 |
+--------+-------+----------+
| No tip |    39 |       23 |
+--------+-------+----------+


In [59]:
gt_tc = evaler.tip_carina_distance(gt=True)
pred_tc = evaler.tip_carina_distance(gt=False)

Processing groundtruth label...
Tip not exist for image  4425991
Tip not exist for image  4425993
Tip not exist for image  4426004
Tip not exist for image  4426032
Tip not exist for image  4426049
Tip not exist for image  4426081
Tip not exist for image  4426112
Tip not exist for image  4426173
Tip not exist for image  4426183
Tip not exist for image  4426195
Tip not exist for image  4426251
Tip not exist for image  4426254
Tip not exist for image  4426280
Tip not exist for image  4426303
Tip not exist for image  4426344
Tip not exist for image  4426353
Tip not exist for image  4426392
Tip not exist for image  4426418
Tip not exist for image  4426419
Tip not exist for image  4426420
Tip not exist for image  4426421
Tip not exist for image  4426422
Tip not exist for image  4426423
Tip not exist for image  4426424
Tip not exist for image  4426425
Tip not exist for image  4426426
Tip not exist for image  4426427
Tip not exist for image  4426428
Tip not exist for image  4426429
Tip not exi

In [60]:
evaler.mean_squared_error(gt_tc, pred_tc)

0.6115582390830152

In [61]:
evaler.calculate_r_squared(gt_tc, pred_tc)

0.8655423503665611